In [1]:
import pandas as pd
import os
import numpy as np
import datetime
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/sales_Margret_20190717'

In [2]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [3]:
all_BL_weekly_sales_file_2018=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
all_BL_weekly_sales_file_2018=sorted([x for x in all_BL_weekly_sales_file_2018 if ("SalesWeekly" in x) & (".txt" in x)])

all_BL_weekly_sales_file_2019=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
all_BL_weekly_sales_file_2019=sorted([x for x in all_BL_weekly_sales_file_2019 if ("SalesWeekly" in x) & (".txt" in x)])

all_BL_weekly_sales_file_2020=list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/"))
all_BL_weekly_sales_file_2020=sorted([x for x in all_BL_weekly_sales_file_2020 if ("SalesWeekly" in x) & (".txt" in x)])


In [4]:
df_sales_2018_19=pd.DataFrame()
for file in all_BL_weekly_sales_file_2018+all_BL_weekly_sales_file_2019+all_BL_weekly_sales_file_2020:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','week_end_dt','gross_sales_amt'])
    df['gross_sales_amt']=df['gross_sales_amt'].astype(float)
    df=df.drop_duplicates()
    
    df['store_type']=np.where(df['location_id']=="6990","Ecomm","Instore")
    df=df.groupby(["week_end_dt","store_type"])['gross_sales_amt'].sum().to_frame().reset_index()
    df_sales_2018_19=df_sales_2018_19.append(df)
    
df_sales_2018_19.shape

(222, 3)

In [5]:
df_sales_2018_19.shape

(222, 3)

In [6]:
df_sales_2018_19['week_end_dt'].nunique()

111

In [7]:
all_BL_weekly_sales_file_2017=list(recursive_file_gen("/home/jian/BigLots/2017_by_weeks/"))
all_BL_weekly_sales_file_2017=[x for x in all_BL_weekly_sales_file_2017 if ("SalesWeekly" in x) & (".txt" in x)]
previous_sales_files=['/home/jian/BigLots/MediaStormDataExtract/MediaStorm Data Extract - Detail Store Sales1 (stores up to 1000).txt',
                     '/home/jian/BigLots/MediaStormDataExtract/MediaStorm Data Extract - Detail Store Sales2 (stores 1001 to 1450).txt',
                     '/home/jian/BigLots/MediaStormDataExtract/MediaStorm Data Extract - Detail Store Sales3 (stores 1451 to 1750).txt',
                     '/home/jian/BigLots/MediaStormDataExtract/MediaStorm Data Extract - Detail Store Sales4 (stores 1751 to 2000).txt',
                     '/home/jian/BigLots/MediaStormDataExtract/MediaStorm Data Extract - Detail Store Sales5 (stores 2001 to 4500).txt',
                     '/home/jian/BigLots/MediaStormDataExtract/MediaStorm Data Extract - Detail Store Sales6 (stores 4501 to 5000).txt',
                     '/home/jian/BigLots/MediaStormDataExtract/MediaStorm Data Extract - Detail Store Sales7 (stores 5001 to 6990).txt'
                     ]

sales_files_2017_May=['/home/jian/BigLots/2017-05/MediaStorm Data Extract - Detail Store Sales May 2017.txt']
sales_files_2017_June_Aug=['/home/jian/BigLots/MediaStorm Data Extract June-August 2017/MediaStorm Data Extract - Detail Store Sales June-August 2017.txt']

all_2017_files=previous_sales_files+sales_files_2017_May+sales_files_2017_June_Aug+all_BL_weekly_sales_file_2017
                           
len(all_2017_files)

26

In [8]:
df_sales_2017=pd.DataFrame()
for file in all_2017_files:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','week_end_dt','gross_sales_amt'])
    df['gross_sales_amt']=df['gross_sales_amt'].astype(float)
    df=df[df['week_end_dt']>="2017-01-01"]
    df=df.drop_duplicates()
    df['store_type']=np.where(df['location_id']=="6990","Ecomm","Instore")
    df_sales_2017=df_sales_2017.append(df)
    
df_sales_2017=df_sales_2017.groupby(["week_end_dt","store_type"])['gross_sales_amt'].sum().to_frame().reset_index()
df_sales_2017=df_sales_2017.sort_values("week_end_dt")
df_sales_2017.head(5)

,week_end_dt,store_type,gross_sales_amt
0,2017-01-07,Ecomm,432202.66
1,2017-01-07,Instore,71550131.95
2,2017-01-14,Ecomm,211334.39
3,2017-01-14,Instore,71815622.84
4,2017-01-21,Ecomm,133178.15


In [9]:
df_sales_2017.shape

(104, 3)

In [10]:
df_sales_2017['week_end_dt'].nunique()

52

In [19]:
all_sales=df_sales_2017.append(df_sales_2018_19)
all_sales=all_sales.sort_values(["store_type","week_end_dt"])
all_sales_table=all_sales.pivot(index="week_end_dt",columns="store_type",values="gross_sales_amt")
all_sales_table=all_sales_table.fillna(0)
all_sales_table['total_sales']=all_sales_table['Ecomm']+all_sales_table['Instore']
all_sales_table=all_sales_table.reset_index()

In [20]:
writer=pd.ExcelWriter('/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/sales_Margret_20190717/sales_by_week_JL_'+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
all_sales_table.to_excel(writer,"sales_by_week",index=False)
all_sales.to_excel(writer,"sales_long",index=False)
writer.save()

In [21]:
all_sales.shape

(326, 3)

In [22]:
all_sales['week_end_dt'].nunique()

163

In [23]:
all_sales_table.shape

(163, 4)

In [25]:
all_sales_table['week_end_dt'].nunique()

163